#### для удобста возьмем датасет (уровнеь дохода человека по его признакам)

### Предсказания уровня дохода человека по выделенным признакам, используя логистическую регрессию и деревья решений.

In [1]:
from warnings import filterwarnings
filterwarnings('ignore')
#импортируем нужные библиотеки
import pandas as pd
import numpy as np

In [2]:
filterwarnings('ignore')

In [4]:
#считаем наш датасет в датафрейм
data = pd.read_csv( 'adult.csv' )
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [5]:
#выведем информацию о датасете
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
age                48842 non-null int64
workclass          48842 non-null object
fnlwgt             48842 non-null int64
education          48842 non-null object
educational-num    48842 non-null int64
marital-status     48842 non-null object
occupation         48842 non-null object
relationship       48842 non-null object
race               48842 non-null object
gender             48842 non-null object
capital-gain       48842 non-null int64
capital-loss       48842 non-null int64
hours-per-week     48842 non-null int64
native-country     48842 non-null object
income             48842 non-null object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [6]:
# выберем признаки на которых будем строить предсказания
selectedColumns = data[ [ 'age', 'hours-per-week', 'education', 'occupation', 'income' ] ]

In [7]:
# переведем значение столбцов education и occupation в значения 0 и 1 и добавим столбцы с соответствующими названиями
X = pd.get_dummies( selectedColumns, columns = [ 'education', 'occupation' ] )

In [8]:
#Поссмотрим каким стал датафрейм
X.head(10)

,age,hours-per-week,income,education_10th,education_11th,education_12th,education_1st-4th,education_5th-6th,education_7th-8th,education_9th,...,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving
0,25,40,<=50K,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,38,50,<=50K,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,28,40,>50K,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,44,40,>50K,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,18,30,<=50K,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,34,30,<=50K,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
6,29,40,<=50K,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,63,32,>50K,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8,24,40,<=50K,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9,55,10,<=50K,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# столбец income мы будем предсказывать, поэтому удалим его из выборки
del X['income']

In [10]:
#Имортируем библиотеку для кодирования значений столбца income
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
X.head()

,age,hours-per-week,education_10th,education_11th,education_12th,education_1st-4th,education_5th-6th,education_7th-8th,education_9th,education_Assoc-acdm,...,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving
0,25,40,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,38,50,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,28,40,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
3,44,40,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,18,30,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
encoder.fit( data['income'] )

LabelEncoder()

In [12]:
#Поссмотрим какие значения (классы)
encoder.classes_

array(['<=50K', '>50K'], dtype=object)

In [13]:
#переведем текущие значения в 0 и 1
encoder.transform( [ '<=50K', '>50K' ] )

array([0, 1])

In [14]:
#выделим все метки из датафрема и присфоим их y
y = pd.Series( data = encoder.transform( data['income'] ) )

In [15]:
y.head(10)

0    0
1    0
2    1
3    1
4    0
5    0
6    0
7    1
8    0
9    0
dtype: int64

In [16]:
#импортируем модуль LogisticRegression
from sklearn.linear_model import LogisticRegression
log_reg_model = LogisticRegression()

In [17]:
#с помощью модуля train_test_split разобьем нашу выборку на тестовую и тренировчную в соотношении 20/80
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [18]:
#обучим наш класификатор на тренировочных данных
log_reg_model.fit( X_train, y_train )

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [19]:
#попробуем предсказать значения y по тестовым данным
log_reg_model.predict(X_test)

array([0, 0, 0, ..., 0, 0, 1])

In [20]:
#можно поссмотреть вероятность угадывания
log_reg_model.predict_proba( X_test )

array([[0.91235456, 0.08764544],
       [0.91265102, 0.08734898],
       [0.54765613, 0.45234387],
       ...,
       [0.74951178, 0.25048822],
       [0.97890562, 0.02109438],
       [0.35388485, 0.64611515]])

In [21]:
#выведем точность нашей модели 
log_reg_model.score(X_test, y_test)

0.7991606100931518

In [22]:
pred = log_reg_model.predict(X_test)

## Попробуем использовать модель деревья решений.

In [24]:
from sklearn.tree import DecisionTreeClassifier

In [41]:
clf_trees = DecisionTreeClassifier(max_depth=2)

In [42]:
clf_trees.fit( X_train, y_train )

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [43]:
#попробуем предсказать значения y по тестовым данным
clf_trees.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0])

In [44]:
#можно поссмотреть вероятность угадывания
clf_trees.predict_proba( X_test )

array([[0.75231406, 0.24768594],
       [0.89073191, 0.10926809],
       [0.75231406, 0.24768594],
       ...,
       [0.89073191, 0.10926809],
       [0.990134  , 0.009866  ],
       [0.75231406, 0.24768594]])

In [45]:
#выведем точность нашей модели 
clf_trees.score(X_test, y_test)

0.7655850138192241

### небольшое наблюдение.... если использовать больше глубину дерева, то точность выше.